1. Import các thư viện cần thiết:

In [ ]:
import torch
import torch.nn as nn

seed = 1
torch.manual_seed(seed)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import unidecode

nltk.download('stopwords ')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from torch.utils.data import Dataset , DataLoader
from sklearn.model_selection import train_test_split

2. Tải bộ dữ liệu: Các bạn tải bộ dữ liệu tại đây. Ở đây, ta sẽ chỉ quan tâm đến file
all_data.csv. Dưới đây là thông tin 4 hàng đầu tiên của bảng dữ liệu:

3. Đọc bộ dữ liệu: Sử dụng thư viện pandas, các bạn đọc bộ dữ liệu lên như sau, lưu ý cần
phải cài đặt tham số encoding='ISO-8859-1' để đưa về định dạng đúng như đoạn code như
sau:

In [ ]:
dataset_path = 'dataset/all -data.csv'
headers = ['sentiment ', 'content ']
df = pd.read_csv(
dataset_path ,
names=headers ,
encoding='ISO -8859 -1'
)

Từ đây, ta có thể xác định được danh sách các class của bài toán bằng phương thức unique()
của Pandas, đồng thời ta cũng sẽ đổi class dạng string thành số nguyên đại diện cho ID của
chúng:

In [ ]:
classes = {
class_name: idx for idx , class_name in enumerate(df['sentiment ']. unique ().tolist ())
}
df['sentiment '] = df['sentiment ']. apply(lambda x: classes[x])

4. Tiền xử lý dữ liệu: Để tận dùng hàm apply của pandas, ta sẽ tiền xử lý văn bản cột
content ngay tại đây. Đầu tiên, định nghĩa hàm chuẩn hóa, nhận tham số đầu vào là 1 text
và trả về 1 text đã được chuẩn hóa:

In [ ]:
english_stop_words = stopwords.words('english ')
stemmer = PorterStemmer ()

def text_normalize(text):
text = text.lower()
text = unidecode.unidecode(text)
text = text.strip()
text = re.sub(r'[^\w\s]', '', text)
text = ' '.join([word for word in text.split(' ') if word not inenglish_stop_words ])
text = ' '.join([ stemmer.stem(word) for word in text.split(' ')])

return text

5. Xây dựng bộ từ vựng: Để huấn luyện dữ liệu văn bản, ta cần chuyển đổi các văn bản
thành vector. Phương thức đơn giản nhất, ta sẽ thu thập toàn bộ các từ trong bộ dữ liệu
thành một tập từ vựng, mỗi từ có một ID riêng. Từ đó, với một văn bản đầu, ta quy đổi
sang ID tương ứng, như vậy sẽ được một vector số nguyên. Đoạn code tạo bộ từ vựng được
thực hiện như sau:

In [ ]:
vocab = []
for sentence in df['content ']. tolist ():
tokens = sentence.split ()
for token in tokens:
if token not in vocab:
vocab.append(token)

vocab.append('UNK')
vocab.append('PAD')
word_to_idx = {word: idx for idx , word in enumerate(vocab)}
vocab_size = len(vocab)

Trong đó, 'UNK' đại diện cho các từ không thuộc bộ từ vựng và 'PAD' đại diện cho các ô
trống được thêm vào để thỏa mãn độ dài tối thiểu của một văn bản mà ta quy định về sau.
Với bộ từ vựng trên, ta xây dựng hàm transform() dùng để chuyển đổi văn bản thành danh
sách các số nguyên như sau:

In [ ]:
def transform(text , word_to_idx , max_seq_len):
tokens = []
for w in text.split():
try:
w_ids = word_to_idx[w]
except:
w_ids = word_to_idx['UNK']
tokens.append(w_ids)

if len(tokens) < max_seq_len:
tokens += [word_to_idx['PAD']] * (max_seq_len - len(tokens))
elif len(tokens) > max_seq_len:
tokens = tokens [: max_seq_len]

return tokens

6. Chia bộ dữ liệu train, val, test: Ta dùng hàm train_test_split của thư viện scikit-learn
để chia bộ dữ liệu thành 3 bộ train/val/test theo tỷ lệ 7:2:1:

In [ ]:
val_size = 0.2
test_size = 0.125
is_shuffle = True
texts = df['content ']. tolist ()
labels = df['sentiment ']. tolist ()

X_train , X_val , y_train , y_val = train_test_split(
texts , labels ,
test_size=val_size ,
random_state=seed ,
shuffle=is_shuffle
)

X_train , X_test , y_train , y_test = train_test_split(
X_train , y_train ,
test_size=val_size ,
random_state=seed ,
shuffle=is_shuffle
)

7. Xây dựng pytorch datasets: Ta triển khai class tên FinancialNews với đầu vào gồm cặp
dữ liệu đầu vào X - y, bộ từ điển cũng như hàm transform như sau:

In [ ]:
class FinancialNews(Dataset):
def __init__(
self ,
X, y,
word_to_idx ,
max_seq_len ,
transform=None
):
self.texts = X
self.labels = y
self.word_to_idx = word_to_idx
self.max_seq_len = max_seq_len
self.transform = transform

def __len__(self):
return len(self.texts)

def __getitem__(self , idx):
text = self.texts[idx]
label = self.labels[idx]

if self.transform:
text = self.transform(
text ,
self.word_to_idx ,
self.max_seq_len
)
text = torch.tensor(text)

return text , label

8. Khai báo dataloader:

In [ ]:
max_seq_len = 32

train_dataset = FinancialNews(
X_train , y_train ,
word_to_idx=word_to_idx ,
max_seq_len=max_seq_len ,
transform=transform
)
val_dataset = FinancialNews(
X_val , y_val ,
word_to_idx=word_to_idx ,
max_seq_len=max_seq_len ,
transform=transform
)
test_dataset = FinancialNews(
X_test , y_test ,
word_to_idx=word_to_idx ,
max_seq_len=max_seq_len ,
transform=transform
)

train_batch_size = 128
test_batch_size = 8

train_loader = DataLoader(
train_dataset ,
batch_size=train_batch_size ,
shuffle=True
)
val_loader = DataLoader(
val_dataset ,
batch_size=test_batch_size ,
shuffle=False
)
test_loader = DataLoader(
test_dataset ,
batch_size=test_batch_size ,
shuffle=False
)

9. Xây dựng mô hình: Chúng ta sẽ xây dựng phân loại cảm xúc với 2 layer RNNs. Vector
tại hidden state cuối cùng của RNN layer thứ 2 sẽ được đưa vào FC layer để thực hiện dự
đoán. Các bạn có thể quan sát rõ hơn thông qua hình dưới đây:<br>
Như vậy, code cài đặt như sau:

In [ ]:
class SentimentClassifier(nn.Module):
def __init__(
self , vocab_size , embedding_dim ,
hidden_size , n_layers , n_classes ,
dropout_prob
):
super(SentimentClassifier , self).__init__ ()
self.embedding = nn.Embedding(vocab_size , embedding_dim)
self.rnn = nn.RNN(embedding_dim , hidden_size , n_layers , batch_first=True)
self.norm = nn.LayerNorm(hidden_size)
self.dropout = nn.Dropout(dropout_prob)
self.fc1 = nn.Linear(hidden_size , 16)
self.relu = nn.ReLU()
self.fc2 = nn.Linear (16, n_classes)

def forward(self , x):
x = self.embedding(x)
x, hn = self.rnn(x)
x = x[:, -1, :]
x = self.norm(x)
x = self.dropout(x)
x = self.fc1(x)
x = self.relu(x)
x = self.fc2(x)

return x

Ta định nghĩa class với tên SentimentClassifier, nhận tham số đầu vào gồm: kích thước bộ
từ vựng (vocab_size), kích thước không gian vector của mỗi từ trong chuỗi đầu vào (em-
bedding_dim), kích thước không gian vector của hidden state (hidden_size), số lượng RNN
layer (n_layers), số class dự đoán của bài toán (n_classes) và tỉ lệ dropout (dropout_prob).
Với class trên, ta khai báo mô hình SentimentClassifier:

In [ ]:
n_classes = len(list(classes.keys()))
embedding_dim = 64
hidden_size = 64
n_layers = 2
dropout_prob = 0.2
device = 'cuda' if torch.cuda.is_available () else 'cpu'

model = SentimentClassifier(
vocab_size=vocab_size ,
embedding_dim=embedding_dim ,
hidden_size=hidden_size ,
n_layers=n_layers ,
n_classes=n_classes ,
dropout_prob=dropout_prob
).to(device)

10. Cài đặt hàm loss và optimizer: Ta sử dụng hàm loss crossentropy cho bài toán phân loại
và Adam optimizer.

In [ ]:
lr = 1e-4
epochs = 50

criterion = nn.CrossEntropyLoss ()
optimizer = torch.optim.Adam(
model.parameters (),
lr=lr
)

11. Thực hiện huấn luyện: Ta định nghĩa hàm fit() và evaluate() dùng để huấn luyện và đánh
giá mô hình như sau:

In [ ]:
def fit(
model ,
train_loader ,
val_loader ,
criterion ,
optimizer ,
device ,
epochs
):
train_losses = []
val_losses = []

for epoch in range(epochs):
batch_train_losses = []

model.train()
for idx , (inputs , labels) in enumerate(train_loader):
inputs , labels = inputs.to(device), labels.to(device)

optimizer.zero_grad ()
outputs = model(inputs)
loss = criterion(outputs , labels)
loss.backward ()
optimizer.step()

batch_train_losses.append(loss.item())

train_loss = sum(batch_train_losses) / len(batch_train_losses)
train_losses.append(train_loss)

val_loss , val_acc = evaluate(
model , val_loader ,
criterion , device
)
val_losses.append(val_loss)

print(f'EPOCH {epoch + 1}:\ tTrain loss: {train_loss :.4f}\tVal loss: {val_loss :.4f}')

return train_losses , val_losses

In [ ]:
def evaluate(model , dataloader , criterion , device):
model.eval()
correct = 0
total = 0
losses = []
with torch.no_grad ():
for inputs , labels in dataloader:
inputs , labels = inputs.to(device), labels.to(device)
outputs = model(inputs)
loss = criterion(outputs , labels)
losses.append(loss.item())
_, predicted = torch.max(outputs.data , 1)
total += labels.size (0)
correct += (predicted == labels).sum().item()

loss = sum(losses) / len(losses)
acc = correct / total

return loss , acc

Tổng hợp tất cả các dữ kiện trên, ta tiến hành huấn luyện mô hình SentimentClassifier:

In [ ]:
train_losses , val_losses = fit(
model ,
train_loader ,
val_loader ,
criterion ,
optimizer ,
device ,
epochs
)

12. Đánh giá mô hình: Sử dụng hàm evaluate() đã xây dựng ở trên, ta đánh giá mô hình trên
hai tập val và test như sau:

In [ ]:
val_loss , val_acc = evaluate(
model ,
val_loader ,
criterion ,
device
)
test_loss , test_acc = evaluate(
model ,
test_loader ,
criterion ,
device
)

print('Evaluation on val/test dataset ')
print('Val accuracy: ', val_acc)
print('Test accuracy: ', test_acc)

13. Triển khai mô hình: Dựa trên mô hình đã huấn luyện và thư viện streamlit để triển khai
ứng dụng:<br>
– Link Streamlit<br>
– Github<br>
– Giao Diện